In [1]:
import os
import sys
import pandas as pd
import openpyxl

import time
from timeit import default_timer as timer
from datetime import datetime
from datetime import datetime, date, timedelta
from bs4 import BeautifulSoup

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.chrome.options import Options


from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# NAVER BLOG Crawling
- title/url 

- ver02-1

In [3]:
star_time = timer()

# 검색어 입력
# query = input("검색어 : ")
# location = str(query)

f = open("crawling/station.txt", 'r', encoding='utf-8')
keywords = f.readlines()
keywords_data = []

# 조회 기간 설정 : 7일(days에서 수정 가능)
# startDate=7일전 날짜, endDate=오늘 날짜
date = datetime.now()
startDate= (date+timedelta(days=-7)).strftime('%Y-%m-%d')
endDate = (datetime.now()).strftime('%Y-%m-%d')

# print(startDate, endDate)

for query in keywords : 
    start_crawling_time = timer()
    print(start_crawling_time)

    # Chrome driver 환경설정 및 실행
    options = webdriver.ChromeOptions()
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_argument('headless')
    driver = webdriver.Chrome(options = options)
    base_url = f"https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=WEEK&orderBy=sim&startDate={startDate}&endDate={endDate}&keyword={query}"
    driver.get(base_url)


    # 총 검색 결과
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    search_number = soup.find(class_="search_number")
    # print("총 블로그글 개수 : ", search_number.text)


    # 연관 검색어 
    sub_keywords_data = soup.select('div.area_keyword > div.list')
    sub_keywords = []
    for i in sub_keywords_data: 
        sub_keywords.append(i.get_text())
    # print("연관검색어:",*sub_keywords)


    # blog url/title 가져오기
    total_num = search_number.text.replace("건",'').replace(",",'')
    total_num = int(total_num)
    end_page=total_num//7+1
    # print(end_page)

    crawling_blog_data = [] 

    try:
        for page_num in range(1,end_page):
            search_url = f"https://section.blog.naver.com/Search/Post.naver?pageNo={page_num}&rangeType=WEEK&orderBy=sim&startDate={startDate}&endDate={endDate}&keyword={query}"
            driver.get(search_url)
            time.sleep(0.1)
            # print(f'{page_num}번째 페이지 접속')

            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")
            # print('html 가져오기 완료')

            # 블로그 제목, url, 포스팅 날짜 가져오기
            posts_data = soup.select('div.info_post > div.desc > a.desc_inner')
            # print('posts_data 가져오기 완료')
            posts_date_data = soup.select('div.info_post > div.writer_info')
            # print('posts_date_data 가져오기 완료')

            for post, post_date in zip(posts_data, posts_date_data):
                while len(posts_data) ==7 :
                    title = post.text.replace('\n','').strip()
                    href = post.attrs['href']
                    date = post_date.find('span','date').text.strip()# Corrected this line
                    try : 
                        date_timestamp = datetime.strptime(date, '%Y. %m. %d.').strftime('%Y-%m-%d')
                    except: 
                        date_timestamp = endDate
                    crawling_blog_data.append({"location": query, "title": title, "url": href, "posted_date": date_timestamp})
                # print("리스트 담기 완료")

    except Exception as e:
                print(f"error : {page_num}페이지 - {e}")
    end_crawling_time = timer()
    print(end_crawling_time)
    print("working_crawling_time: {} sec".format(end_crawling_time-start_crawling_time))

# print("carwling data :",crawling_blog_data)
driver.close()
end_time = timer()
print("working_time: {} sec".format(end_time-star_time))

61937.0152683


In [3]:
len(crawling_blog_data)

1351

In [5]:
df = pd.DataFrame(crawling_blog_data)
df


,location,title,url,posted_date
0,가산디지털단지역,가산 SH드림타워 분양권해지 분양계약해지 가산디지털단지역 지식산업센터,https://blog.naver.com/kya7702/223454718395,2024-05-22
1,가산디지털단지역,가산디지털단지역 맛집 육즙가득했던 돼지목살 맛집,https://blog.naver.com/hajin5423/223452745915,2024-05-20
2,가산디지털단지역,"육즙 대박 삼겹살 맛집, 가산디지털단지고기집 ""육뜸 가디역""",https://blog.naver.com/ddbbddbb12/223452987978,2024-05-20
3,가산디지털단지역,"[가산디지털단지역] 즉석떡볶이 점심맛집, 중독",https://blog.naver.com/chloe_yaaa/223453263399,2024-05-21
4,가산디지털단지역,[가산디지털단지맛집] 산더미같은 가산물갈비&즉석소불고기,https://blog.naver.com/ayddl22/223454189015,2024-05-21
...,...,...,...,...
1346,가산디지털단지역,청소사례,https://blog.naver.com/k771030/223451449111,2024-05-19
1347,가산디지털단지역,까치산역원룸텔 픽셀하우스 신까치산점 소개해드릴게요.,https://blog.naver.com/happier89/223449994039,2024-05-17
1348,가산디지털단지역,을지로 쌀국수 '벱비엣' 을지로 본점 방문 후기,https://blog.naver.com/kong_ss_cat/223449388194,2024-05-19
1349,가산디지털단지역,[리치온 독서모임]06권.부동산 변곡점이 왔다 by 삼토시,https://blog.naver.com/wyh6042/223452544924,2024-05-20


In [6]:
df1 = df.query('posted_date.str.contains("전")')
df1['posted_date'] = endDate
# df1.sort_values('posted_date', axis=0, ascending=False, inplace=True) 
df1

C:\Users\20061\AppData\Local\Temp\ipykernel_22112\2370409416.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['posted_date'] = endDate


,location,title,url,posted_date
5,가산디지털단지역,가산디지털단지역 맛집 신선한 소생갈비살 맛집 놀부자정육,https://blog.naver.com/wd5linda/223456157169,2024-05-23
17,가산디지털단지역,"가산디지털단지역 맛집 돈마카세 :: 순만 한우,한돈 전문점",https://blog.naver.com/iamjieun2/223455352325,2024-05-23
20,가산디지털단지역,"[가산디지털단지역 전세] G밸리 인근, 버팀목 안심 신축 투룸",https://blog.naver.com/wlqrhdfir/223455098054,2024-05-23
23,가산디지털단지역,가산디지털단지역 맛집 여신족발 처음 먹어본 뒷발 후기,https://blog.naver.com/hyunhee_37/223448230299,2024-05-23
26,가산디지털단지역,가산디지털단지역 역세권 지식산업센터 IT미래타워 사무실 임대,https://blog.naver.com/bdrsquare/223454772709,2024-05-23
...,...,...,...,...
1333,가산디지털단지역,독산맛집 아낌없이 주는 곱창 맛집 문정곱창,https://blog.naver.com/jjoyou123/223456047298,2024-05-23
1334,가산디지털단지역,회식장소로좋은 독산맛집 문정곱창 막창 대창 염통 전골 좋은재료라 맛이대박...,https://blog.naver.com/richs0/223453582560,2024-05-23
1338,가산디지털단지역,저평가 아파트 동작협성휴포레시그니처 역까지 거리 및 층간소음,https://blog.naver.com/2013check/223455983991,2024-05-23
1341,가산디지털단지역,금천구발마사지 피로 제대로 풀고 온 타이라이프,https://blog.naver.com/bogobogossg/223455433798,2024-05-23


In [7]:
df2 = df.query('not posted_date.str.contains("시간 전")')
df2.sort_values('posted_date', axis=0, ascending=False, inplace=True) 
df2

C:\Users\20061\AppData\Local\Temp\ipykernel_22112\182116724.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.sort_values('posted_date', axis=0, ascending=False, inplace=True)


,location,title,url,posted_date
586,가산디지털단지역,살 빠지는 음식 /구로디지털단지/구디PT/PT피트니스/에이펙스피트니스,https://blog.naver.com/apexfitness02/223456238892,41분 전
99,가산디지털단지역,"[가산디지털단지역 맛집] 코미오 / 일본식 소고기 구이 니꾸세트와 라멘, 우설 첫 도전",https://blog.naver.com/sudary-/223456242154,37분 전
0,가산디지털단지역,가산 SH드림타워 분양권해지 분양계약해지 가산디지털단지역 지식산업센터,https://blog.naver.com/kya7702/223454718395,2024-05-22
613,가산디지털단지역,[단지소개]천왕역 모아엘가 트레뷰 분양권 (7호선 30평대 신축 가격 실화??)...,https://blog.naver.com/dreammakerss/223454887296,2024-05-22
596,가산디지털단지역,"[가디역 원룸] 저렴한 월세에 깔끔한 내부, 주변 직장인분들은 좋겠네요",https://blog.naver.com/wlqrhdfir/223454782445,2024-05-22
...,...,...,...,...
439,가산디지털단지역,"구로디지털단지역 지원시설 80평 사무실 임대, 대륭포스트타워8차 첫입주 하실 수...",https://blog.naver.com/hjbv/223448759999,2024-05-16
927,가산디지털단지역,합정카페 [79파운야드 합정점] 테라스가 있는 크로플 디저트맛집,https://blog.naver.com/dusghk2531/223448800836,2024-05-16
444,가산디지털단지역,구로디지털단지 선글라스 #안경공간 ::: 여름맞이 #뉴진스선글라스 카린(Carin)☀️,https://blog.naver.com/happynudge_official/223...,2024-05-16
930,가산디지털단지역,임금 또는 퇴직금 지급의무의 존부 및 범위에 과하여 다툴 만한 근거가 있는 경우,https://blog.naver.com/lawyerjym/223448311741,2024-05-16


In [8]:
total_df = pd.concat([df1,df2], ignore_index=True)
total_df

,location,title,url,posted_date
0,가산디지털단지역,가산디지털단지역 맛집 신선한 소생갈비살 맛집 놀부자정육,https://blog.naver.com/wd5linda/223456157169,2024-05-23
1,가산디지털단지역,"가산디지털단지역 맛집 돈마카세 :: 순만 한우,한돈 전문점",https://blog.naver.com/iamjieun2/223455352325,2024-05-23
2,가산디지털단지역,"[가산디지털단지역 전세] G밸리 인근, 버팀목 안심 신축 투룸",https://blog.naver.com/wlqrhdfir/223455098054,2024-05-23
3,가산디지털단지역,가산디지털단지역 맛집 여신족발 처음 먹어본 뒷발 후기,https://blog.naver.com/hyunhee_37/223448230299,2024-05-23
4,가산디지털단지역,가산디지털단지역 역세권 지식산업센터 IT미래타워 사무실 임대,https://blog.naver.com/bdrsquare/223454772709,2024-05-23
...,...,...,...,...
1349,가산디지털단지역,"구로디지털단지역 지원시설 80평 사무실 임대, 대륭포스트타워8차 첫입주 하실 수...",https://blog.naver.com/hjbv/223448759999,2024-05-16
1350,가산디지털단지역,합정카페 [79파운야드 합정점] 테라스가 있는 크로플 디저트맛집,https://blog.naver.com/dusghk2531/223448800836,2024-05-16
1351,가산디지털단지역,구로디지털단지 선글라스 #안경공간 ::: 여름맞이 #뉴진스선글라스 카린(Carin)☀️,https://blog.naver.com/happynudge_official/223...,2024-05-16
1352,가산디지털단지역,임금 또는 퇴직금 지급의무의 존부 및 범위에 과하여 다툴 만한 근거가 있는 경우,https://blog.naver.com/lawyerjym/223448311741,2024-05-16


In [29]:
file_name = f"naver_crawling_{query}_{startDate}-{endDate}.csv"
total_df.to_csv(file_name, index=False)
data = pd.read_csv(file_name)
data

,location,title,url,posted_date
0,회기역,회기역 월주차 2번출구 인근 편리한곳,https://blog.naver.com/plqh9295/223455804000,2024-05-23
1,회기역,"회기동 월주차 _ 회기역 도보 5분거리의 지정석,단일 주차장",https://blog.naver.com/zoomansa1/223455635624,2024-05-23
2,회기역,"행복주택 예비입주자 청약 신청 총정리 (월세 6만원부터,초역세권, 가좌역/회기역)",https://blog.naver.com/newchorok/223455224162,2024-05-23
3,회기역,회기역 맛집 회기왕갈비탕 점심 혼밥하기 좋은 외대 맛집,https://blog.naver.com/kwakyoukyeong/223455354721,2024-05-23
4,회기역,회기역 근처 맛집 밥집 '큐큐면관' - 5년 거주민이 데려가준 곳,https://blog.naver.com/joppdbsk/223450702877,2024-05-23
...,...,...,...,...
416,회기역,동대문구 실용음악학원 동대문구 드럼학원 동대문구 보컬학원 동대문구 댄스학원...,https://blog.naver.com/fwinds/223448265584,2024-05-16
417,회기역,24년 4월 일기(부동산 임장/초보 체험단/ADSP 독학/프리미엄 막걸리),https://blog.naver.com/nongnogne/223444593219,2024-05-16
418,회기역,경희대카페 스터디하기좋은 대형카페 _ 8번가 8street,https://blog.naver.com/mydreameruza/223448746614,2024-05-16
419,회기역,동대문미용실 악성곱슬 볼륨매직 예스헤어 만족합니다,https://blog.naver.com/co1823/223447979677,2024-05-16


- ver02-2

In [ ]:
# 검색어 입력
# query = input("검색어 : ")
# location = str(query)

f = open("crawling/station.txt", 'r', encoding='utf-8')
keywords = f.readlines()
keywords_data = []

# 조회 기간 설정 : 7일(days에서 수정 가능)
# startDate=7일전 날짜, endDate=오늘 날짜
date = datetime.now()
startDate= (date+timedelta(days=-7)).strftime('%Y-%m-%d')
endDate = (datetime.now()).strftime('%Y-%m-%d')

# 브라우저 옵션 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument('headless')

# 브라우저 초기화
driver = webdriver.Chrome(options=options)

for query in keywords:
    try:
        # 검색 URL 생성
        base_url = f"https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=WEEK&orderBy=sim&startDate={startDate}&endDate={endDate}&keyword={query}"
        driver.get(base_url)
        time.sleep(1)  # 페이지가 로드될 때까지 대기

        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        search_number = soup.find(class_="search_number")
        total_num = int(search_number.text.replace("건",'').replace(",",''))

        # 페이지마다 게시물 가져오기
        for page_num in range(1, total_num // 7 + 2):
            search_url = f"https://section.blog.naver.com/Search/Post.naver?pageNo={page_num}&rangeType=WEEK&orderBy=sim&startDate={startDate}&endDate={endDate}&keyword={query}"
            driver.get(search_url)
            time.sleep(1)  # 페이지가 로드될 때까지 대기

            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")

            # 블로그 포스트 데이터 가져오기
            posts_data = soup.select('div.info_post > div.desc > a.desc_inner')
            posts_date_data = soup.select('div.info_post > div.writer_info')

            # 타이틀 수가 7개가 아니면 루프를 빠져나오기
            if len(posts_data) == 7:
                for post, post_date in zip(posts_data, posts_date_data):
                    title = post.text.replace('\n', '').strip()
                    href = post.attrs['href']
                    date = post_date.find('span', 'date').text.strip()
                    if '전' in date or '시간 전' in date:
                        try:
                            date_timestamp = datetime.strptime(date, '%Y. %m. %d.').strftime('%Y-%m-%d')
                        except ValueError:
                            date_timestamp = endDate
                        crawling_blog_data.append({"location": query, "title": title, "url": href, "posted_date": date_timestamp})
            else : 
                break
    except Exception as e:
        print(f"에러 발생: {e}")

# 브라우저 종료
driver.quit()

In [ ]:
total = 25000
person = 3
beer = 5000
taxi = 5300
Jo = total/person+beer
Kim = total/person+beer
Kwon = total/person - taxi

print(f"은별:{Jo}")
print(f"서윤:{Kim}")
print(f"민기:{Kwon}")